In [23]:
%%time
from algoutils import *

raw_courses = read('data/courses.txt')
raw_industries = read('data/industries.txt')
raw_job_industries = read('data/job_industries.txt')

CPU times: user 6.67 ms, sys: 0 ns, total: 6.67 ms
Wall time: 5.19 ms


In [24]:
%%time
# note: enchant module and library should be installed
from cleaning import clean

clean_courses = clean(raw_courses, 'course', 'data/course_dictionary.txt')
clean_industries = clean(raw_industries, 'industry', 'data/industry_dictionary.txt')
clean_job_industries = clean(raw_job_industries, 'industry', 'data/industry_dictionary.txt')

CPU times: user 11.3 s, sys: 10 ms, total: 11.3 s
Wall time: 11.3 s


In [25]:
%%time
# note: nltk module and nltk.wordnet corpus should be installed
from clustering import stem_cluster
 
course_clusters = stem_cluster(clean_courses)
industry_clusters = stem_cluster(clean_industries)
job_industry_clusters = stem_cluster(clean_job_industries, mode=5)

CPU times: user 63.3 ms, sys: 0 ns, total: 63.3 ms
Wall time: 59.5 ms


In [26]:
sorted(course_clusters)[:10]

['ACCOUNTANCY',
 'ACCOUNTING',
 'ADMINISTRATION',
 'ADVERTISING',
 'AGRIBUSINESS',
 'AGRICULTURAL',
 'AGRICULTURE',
 'ANIMATION',
 'APPLIED',
 'ARCHITECTURAL']

In [27]:
sorted(industry_clusters)[:10]

['ACCOUNTING',
 'ADVERTISING',
 'AGENCY',
 'ASSISTANT',
 'AUTOMOTIVE',
 'BANKING',
 'BEAUTY',
 'BPO',
 'BROKERAGE',
 'BUILDING']

In [28]:
sorted(job_industry_clusters)[:10]

['BUSINESS',
 'CALLCENTER',
 'COMPUTER',
 'CONSTRUCTION',
 'DEPARTMENT',
 'DESIGN',
 'DEVELOPMENT',
 'DISTRIBUTION',
 'EDUCATION',
 'ENGINE']

In [29]:
%%time

from analysis import *

# note: pandas and sklearn modules should be installed
# initializes a likelihood data frame based on row and column labels.
# Performs a content-based filtering technique using features.
# vectorizer can be 'tfidf' or 'count'

CourseIndustry = likelihood_matrix(course_clusters, industry_clusters, vectorizer='tfidf', ngram_range=(3,4))

CPU times: user 86.7 ms, sys: 0 ns, total: 86.7 ms
Wall time: 86.2 ms


In [30]:
CourseIndustry.dataframe.head()

,ENGINE,REPAIR,HARDWARE,PRODUCT,PARTS,CALLCENTER,LINE,CAFE,CUSTOMER,OFFICE,...,STORE,BUILDING,CITY,PLANT,PUBLISHING,FABRICATION,HEALTH,ASSISTANT,DISTRIBUTOR,BROKERAGE
LINGUISTICS,0,0,0,0,0,0.000000,0.111031,0,0,0,...,0,0.023,0,0,0.020312,0.000000,0,0.045016,0.042213,0
HARDWARE,0,0,1,0,0,0.000000,0.000000,0,0,0,...,0,0.000,0,0,0.000000,0.000000,0,0.000000,0.000000,0
AGRICULTURAL,0,0,0,0,0,0.000000,0.000000,0,0,0,...,0,0.000,0,0,0.000000,0.052721,0,0.000000,0.000000,0
ELEMENTARY,0,0,0,0,0,0.034037,0.000000,0,0,0,...,0,0.000,0,0,0.000000,0.000000,0,0.000000,0.000000,0
NCII,0,0,0,0,0,0.000000,0.000000,0,0,0,...,0,0.000,0,0,0.000000,0.000000,0,0.000000,0.000000,0


In [31]:
%%time

# note: pandas and sklearn modules should be installed
# derives the change in likelihood if an entry

# computes delta likelihood using the following steps: (non-verbatim)
# clabels = df.column_labels
# rlabels = df.row_labels
# vr = [cos_similarity(ur, label) for label in rlabels]
# vc = [cos_similarity(vr, label) for label in clabels]
# dL = vr * vc^T

d1 = CourseIndustry.delta('CALLCENTE', 'BUSINESS')
d2 = CourseIndustry.delta('MARKETING', 'RESEARCH & DEVELOPMENT')
d3 = CourseIndustry.delta('AGRICULTURE', 'FINANCE')

CPU times: user 123 ms, sys: 3.33 ms, total: 127 ms
Wall time: 124 ms


In [32]:
%%time

# dynamic update of likelihood matrix
# just accumulate small likelihoods
# accounts for relative likelihood and not absolute percentage

users = [
    ('COMPUTER SCIENCE', 'INFORMATION TECHNOLOGY'),
    ('ACCOUNTING', 'BUSINESS'),
    ('BUSINESS ADMINSTRATION', 'FINANCE'),
    ('MARKETING', 'FINANCE'),
    ('MATHEMATICS', 'FINANCE'),
    ('SOCIAL SCIENCE', 'CONSULATION'),
    ('LITERATURE', 'WRITING'),
    ('MANAGEMENT', 'BANKING'),
    ('SCIENCES', 'RESEARCH')
]

for course, industry in users:
    CourseIndustry.add_match(course, industry)

CPU times: user 287 ms, sys: 3.33 ms, total: 290 ms
Wall time: 290 ms


In [33]:
# get specific likelihood coefficient of a pair of clusters
# e.g. likelihood of courses from the BUSINESS industry

# note: 1.00 does not mean 100%
# likelihood metric is relative
# the more data, the more accurate the prediction

%time CourseIndustry.dataframe.BUSINESS.nlargest(10)

CPU times: user 3.33 ms, sys: 0 ns, total: 3.33 ms
Wall time: 1.28 ms


BUSINESS        1.000000
ACCOUNTING      1.000000
AGRIBUSINESS    0.751331
ACCOUNTANCY     0.592993
WRITING         0.164020
DRAFTING        0.139844
MARKETING       0.127857
NURSING         0.110499
ADVERTISING     0.076887
MARINE          0.072553
Name: BUSINESS, dtype: float64

In [34]:
%%time

# transpose data frame for rows
CourseIndustry.dataframe.T.SCIENCE.nlargest(6)

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 1.51 ms


RESEARCH       0.952430
INFORMATION    0.632183
TECHNOLOGY     0.462987
CONSULTING     0.412805
CONSULTANCY    0.370199
EDUCATION      0.258203
Name: SCIENCE, dtype: float64

In [35]:
%%time

# add count from a specific matching pair
# note: this is synonymous to CourseIndustry.dataframe += CourseIndustry.delta('PHYSICS', 'REPAIR SUPPLY')

CourseIndustry.add_match('PHYSICS', 'RESEARCH')

CPU times: user 20 ms, sys: 0 ns, total: 20 ms
Wall time: 19.2 ms


In [36]:
%%time

# returns a vector of recommend likelihood of a course from a specific industry
# since courses are rows, we find column matches
# we can also make the method output percentages

matches = CourseIndustry.find_column_matches('PHYSICS', with_labels = True, percentage = True)

CPU times: user 3.33 ms, sys: 0 ns, total: 3.33 ms
Wall time: 688 µs


In [37]:
# rank recommendations with the highest percentage
matches.nlargest(5)

RESEARCH       55.662739
MEDICAL         5.643027
ELECTRICAL      5.305848
ELECTRONICS     4.931548
FINANCE         4.829725
dtype: float64

In [38]:
%%time

# we can also get the recommendation score of a specific pair
# this is equivalent to SV1^T * LM * SV2^T
# where SV is similarity_vector, LM is likelihood matrix

CourseIndustry.recommendation_score('PHYSICS', 'SCIENTIFIC RESEARCH')

CPU times: user 30 ms, sys: 0 ns, total: 30 ms
Wall time: 27.9 ms


55.353636624354294

In [39]:
# You can also get the dot product of data frames
# to get bridged likelihood

IndustryJob = likelihood_matrix(industry_clusters, job_industry_clusters, vectorizer='tfidf', ngram_range=(3,4))
CourseJob = CourseIndustry * IndustryJob

In [40]:
CourseJob.recommendation_score('PHYSICS', 'MANAGEMENT')

0.28752848943447895

In [41]:
CourseJob.recommendation_score('COMPUTER SCIENCE', 'ARCHITECTURE')

2.0381146684142672

In [42]:
CourseJob.recommendation_score('COMPUTER SCIENCE', 'TECHNOLOGY')

37.428856177544361

In [43]:
# after adding a match to course and a job, recommendation score should go up
CourseJob.add_match('COMPUTER SCIENCE', 'TECHNOLOGY')
CourseJob.add_match('PROGRAMMING', 'COMPUTER')

In [44]:
CourseJob.recommendation_score('PROGRAMMING', 'COMPUTER')

69.022835016757483